In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import chi2

## Simuliere eine Logit Regression

In [ ]:
np.random.seed(245)
Obs=150
Cat= ("A","B","C")
x2 = np.random.choice(Cat, size=Obs)
x1 = np.random.normal(size=Obs)

LinPred= 0.5 +5.4*x1 + -2.5* (x2=="B")+ 2.5*(x2=="C")+ np.random.logistic(size=Obs, scale=3.5)

y= np.zeros(Obs)
y[LinPred >=0] =1

X= np.c_[x1,pd.get_dummies(x2).drop("A",axis=1)]

In [ ]:
logit_base=sm.Logit(y,sm.add_constant(X))
logit=logit_base.fit()
print(logit.summary())

## Wald Test

## Definiere hier die C-Matrix und damit den Hypothesentest

In [ ]:
C=np.array([0,1,0,0,0,0,0,1]).reshape(2,4) # Zwei Koeffizienten simulatan auf 0 testen
#C=np.array([0,1,-1,0]).reshape(1,4) # Differenz zweier Koeffizienten testen
C

In [ ]:
Beta=logit.params
Beta

### 1. Berechne: $C\cdot\beta$

In [ ]:
CB=np.array(C@Beta)
CB

### 2. Extrahiere die Kovaranzmatrix
#### Die Kovarianzmatrix wird mit 10000 skaliert und gerundet um das Rechnen zu erleichtern

$$ Cov(\hat{\beta})= - H(\hat{\beta})^{-1}$$

In [ ]:
Cov=np.round(-np.linalg.inv(logit_base.hessian(Beta))*10000,0)
Cov

### 3. Berechne: $ C \cdot  Cov(\hat{\beta}) \cdot C^{T}$

In [ ]:
temp=C@Cov_temp@C.T/10000
temp

### 4. Berechne: $ [C \cdot  Cov(\hat{\beta}) \cdot C^{T}]^{-1}$

In [ ]:
Inverse=np.linalg.inv(temp)
Inverse

### 5. Berechne die Teststatistik: $ C\beta^{T} [C \cdot  Cov(\hat{\beta}) \cdot C^{T}]^{-1}C\beta$

In [ ]:
t=CB.T@Inverse@CB
t

### 6. Vergleich mit Teststatistik
#### df =1, falls die Differenz getestet wird
#### df >1, falls mehrere Koeffizienten simultan auf 0 getestet werden

In [ ]:
chi2.ppf(0.95, df=2)

In [ ]:
chi2.ppf(0.95, df=1)